# Dynamic Programing

In this notebook, we will develop a dynamic programming solution to a grid world problem. By this I mean a function, called a policy: 

$$ \pi : S \times A \to [0,1] $$

that gives for each state the probability of choosing an action. A policy is optimal if by following this policy(i.e. choosing action $a$ in state $s$ with probability $ \pi(s,a)$) we maximize the expected reward over an episode(here we can think of an episode starting at a random position and ending whenever we reach the terminal state).

For this notebook I will consider a deterministic setting; all actions always have the same effect in any state. We will make free use of the information about the world to construct a model of the world's actions, which is required for dynamic programming. In this sense this not a reinforcement learning approach, as we will not have an agent interacting with the world to discover its behavior.

Under these assumptions, the problem is pretty straightforward, and we will get an optimal solution by using an generalized policy iteration algorithm(GPI).

In [1]:
import sys

sys.path.append("../..")

import pandas as pd
import numpy as np

from grid_world.grid_world import GridWorld
from grid_world.visualization.format_objects import (
    get_policy_rec_str,
    get_policy_eval_str,
    get_world_str,
)
from grid_world.utils.policy import get_policy_rec
from grid_world.action import Action


np.set_printoptions(precision=2)

# Our World

In [2]:
gworld = GridWorld(
    grid_shape=(5, 6),
    terminal_states_coordinates=((0, 5),),
    walls_coordinates=((0, 1), (1, 1), (2, 3), (3, 3)),
    traps_coordinates=((1, 3),),
)
print(get_world_str(gworld))

4                  

3          █       

2          █       

1    █     ☠       

0 ⚐  █           ✘ 

  0  1  2  3  4  5 


This is the world we will be considering, our goal is to reach the termial state as fast as possible, avoiding the trap. If this looks strange to you please refer to the readme file for more details.

# World Modeling

In order to solve this problem with dynamic programming we will need a model of the world and a reward function. Mathematicaly these are the functions we need:

$$ M_w: S \times A \to \mathbb{P}(S) $$
$$ R_w: S \times A \to \mathbb{R} $$

where $M_w$ gives for each pair state action $(s,a)$ a probability distribution over the states $S$, these indicate the probabilitie of moving to this new state when taking action $a$ in state $s$. This means that $M_w(s,a): S \to [0, 1]$ is also a function and $M_w(s,a)(s_0)$ is the probability of getting to $s_0$ when taking action $a$ in state $s$. Since we are in a determinitisct setting these values will be either 0 or 1.

On the hand, $R_w(s,a)$ is the reward we get for taking action $a$ in state $s$. This is something we need to choose by ourselves. Since we want to reach the terminal state as soon as possible we will add a negative reward whenever we take an action outside it, for flavor we will also a big negative reward for being inside the trap. There is no reward for being in the terminal state since this should indicate the end of an episode(this is necessary for our implementation).

In [3]:
# lets make some restrictions on the available actions
actions = [Action.up, Action.down, Action.left, Action.right]


def r(effect):
    if effect == -1:
        return -100
    elif effect == 1:
        return 0
    else:
        return -1


rewards_dict = {
    (s, a): r(gworld.take_action(s, a)[1]) for s in gworld.states for a in actions
}
rewards = lambda x, y: rewards_dict[(x, y)]

In [4]:
def world_model(s, a, world=gworld):
    final_state = world.take_action(s, a)[0]
    return lambda x: 1 if x == final_state else 0

# Policy evaluation

Alright, the first step to implementing a dynamic programming solution is doing policy evaluation, this means that given a policy $\pi$ we want to calculate a function, called the value function of $\pi$:

$$ V_{\pi}: S \to \mathbb{R} $$

that gives for each state $s$ the expected reward we will get until we reach a terminal state, when following our policy(usualy discounted by a $\gamma$ factor which we will set to 1).

There are many ways to calculate this, we will use an iterative approach that can be generalized to the reinforcement learning methods we want to explore. The idea is to start with a random value function $V$ and improve each state estimate like this:

$$ V(s) \leftarrow \sum_{a \in A}\pi(s,a)\sum_{s_0 \in S}M_w(s,a)(s_0)(R(s,a) + \gamma V(s_0))$$

This is essentialy bootstrapping, for each state we of observe the reward for taking action $a$ and the estimated value of our new state, adding these gives an estimate of the value of this action in this state, so we average everything with the respective probability.

Notice that, since $V(s_0)$ is expected to be wrong this new estimate can also be wrong, however now it incorporates information about the actual rewards, and by iterating this method $V$ actually converges to $V_\pi$.

When implementing this we will make $V$ a hashmap(a dictionary) since it is much more practical to update hash values then it is to change functions.

In [5]:
def _acc_V(s, V, pi, world_model, reward_function, states, gamma):
    return np.sum(
        [
            pi(s, a)
            * np.sum(
                [
                    world_model(s, a)(s0) * (reward_function(s, a) + gamma * V[s0])
                    for s0 in states
                ]
            )
            for a in actions
        ]
    )


def _iterate_policy_step(pi, world_model, reward_function, actions, states, V_0, gamma):
    V = V_0.copy()
    for s in states:
        V_0[s] = _acc_V(s, V, pi, world_model, reward_function, states, gamma)
    return np.amax(np.abs([V_0[x] - V[x] for x in V_0]))


def iterative_policy_evalution(
    pi, world_model, reward_function, actions, states, V_0, epsilon=0.01, gamma=1
):
    delta = 2 * epsilon
    while delta > epsilon:
        delta = _iterate_policy_step(
            pi, world_model, reward_function, actions, states, V_0, gamma
        )

    return V_0

Lets test this on a random policy.

In [6]:
def pi(s, a):
    return 0.25


V_0 = {x: 0 for x in gworld.states}
V_pi = iterative_policy_evalution(pi, world_model, rewards, actions, gworld.states, V_0)
print(get_policy_eval_str(V_pi, gworld))

  -773.16    -761.50    -731.95    -662.40    -588.89    -556.93  

  -780.86    -775.42    -767.99               -543.35    -521.00  

  -790.05    -787.36    -792.64               -516.19    -458.75  

  -797.97               -818.60    -901.92    -542.50    -335.08  

  -801.93               -757.28    -692.00    -412.83       0.00  




It's pretty hard to tell whether this is right or not, but it will get clearer latter.

# Policy improvement

The other corner stone of GPI is to improve the policy, this is pretty obvious, once we have values for each state we simply make a policy that will send us to the state with better value; this is called a greedy policy with repect to $V$. 

In [7]:
def q(s, a, V, world_model, reward_function, states):
    return reward_function(s, a) + np.sum(
        [world_model(s, a)(s0) * V[s0] for s0 in states]
    )


# TODO: function needs improvement
def _argmax_q(s, V, world_model, reward_function, actions, states):
    best_score = q(s, actions[0], V, world_model, reward_function, states)
    best_action = actions[0]
    for a in actions:
        qa = q(s, a, V, world_model, reward_function, states)
        if qa > best_score:
            best_score = qa
            best_action = a
    return best_action


def get_greedy_policy(V, world_model, reward_function, actions, states):
    gpr = {
        s: _argmax_q(s, V, world_model, reward_function, actions, states)
        for s in states
    }

    return lambda s, a: 1 if (a == gpr[s]) else 0

In [8]:
pi_1 = get_greedy_policy(V_pi, world_model, rewards, actions, gworld.states)
pr = get_policy_rec(pi_1, gworld, actions)
print(get_policy_rec_str(pr, gworld))

 →  →  →  →  ↓  ↓ 

 ↑  ↑  ↑  █  ↓  ↓ 

 ↑  ↑  ↑  █  →  ↓ 

 ↑  █  ↓  ☠  →  ↓ 

 ↑  █  →  →  →  ✘ 




# GPI - Iterating till convergence

Notice that since we change the policy to say $\pi'$, $V$ may not be a good estimate of $V_{\pi'}$, so $\pi'$ is not necessarily greedy with respect to $V_{\pi'}$. So the idea is to repeat the process until we get a policy that is greedy with respect to its value function; it is not hard to see that this is an optimal policy(check Sutton and Barto if you need).

I will use changes in the value function as the stop criteria, since it is a little easier to check. This also guarantess that the policy is optimal.

In [9]:
max_epochs = 20


def float_dict_compare(d0, d1):
    return np.all([np.isclose(d0[x], d1[x]) for x in d0])


def dpi_step(V_pi, world_model, reward_function, actions, states):
    pi_1 = get_greedy_policy(V_pi, world_model, reward_function, actions, states)
    V_pi_1 = iterative_policy_evalution(
        pi_1, world_model, reward_function, actions, states, V_pi
    )

    return pi_1, V_pi_1


def pi(s, a):
    return 0.25


V_0 = {x: 0 for x in gworld.states}
V_pi = iterative_policy_evalution(pi, world_model, rewards, actions, gworld.states, V_0)

for i in range(max_epochs):
    V_pi_0 = V_pi.copy()
    pi, V_pi = dpi_step(V_pi, world_model, rewards, actions, gworld.states)

    if float_dict_compare(V_pi, V_pi_0):
        print("policy convergerd")
        break

    print(f"epoch: {i}")
    pr = get_policy_rec(pi, gworld, actions)
    print(get_policy_rec_str(pr, gworld))

epoch: 0
 →  →  →  →  ↓  ↓ 

 ↑  ↑  ↑  █  ↓  ↓ 

 ↑  ↑  ↑  █  →  ↓ 

 ↑  █  ↓  ☠  →  ↓ 

 ↑  █  →  →  →  ✘ 


epoch: 1
 →  →  →  →  ↓  ↓ 

 ↑  ↑  ↑  █  ↓  ↓ 

 ↑  ↑  ↓  █  ↓  ↓ 

 ↑  █  ↓  ☠  ↓  ↓ 

 ↑  █  →  →  →  ✘ 


epoch: 2
 →  →  →  →  ↓  ↓ 

 ↑  ↑  ↓  █  ↓  ↓ 

 ↑  →  ↓  █  ↓  ↓ 

 ↑  █  ↓  ☠  ↓  ↓ 

 ↑  █  →  →  →  ✘ 


epoch: 3
 →  →  ↓  →  ↓  ↓ 

 ↑  ↓  ↓  █  ↓  ↓ 

 →  →  ↓  █  ↓  ↓ 

 ↑  █  ↓  ☠  ↓  ↓ 

 ↑  █  →  →  →  ✘ 


epoch: 4
 →  ↓  ↓  →  ↓  ↓ 

 ↓  ↓  ↓  █  ↓  ↓ 

 →  →  ↓  █  ↓  ↓ 

 ↑  █  ↓  ☠  ↓  ↓ 

 ↑  █  →  →  →  ✘ 


policy convergerd


Notice how the policy starts by avoiding the shorter path, which passes near the trap. This happens because the initial random policy has a chance of walking to the trap, and as a consequence the values of states near the trap start much lower then their values under the optimal policy. As we iterate these values get adjusted to the improved policies, and the improvements sorta propagates back to other states.